<a href="https://colab.research.google.com/github/cfperez97/Pothole-Identifier/blob/main/PotholeIdentifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**IMPORTANT: Remember to upload your Kaggle API token**

Install and import any necessary packages

In [ ]:
! pip install kaggle
! pip install ultralytics
import ultralytics
ultralytics.checks()
import os
import re
from pathlib import Path
import shutil
from sklearn.model_selection import train_test_split
from google.colab import drive

Ultralytics YOLOv8.2.2 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 29.1/201.2 GB disk)


Mount your Google Drive to permenantly save your models and results

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


Create, download, and place your Kaggle API token into the folder of your choice

Establish where to look for the Kaggle API token

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

Set the permissions of the Kaggle API token to read/write owner-only to avoid a warning message

In [ ]:
! chmod 600 /content/kaggle.json

Downoad and unzip the dataset

In [ ]:
! kaggle datasets download --unzip felipemuller5/nienaber-potholes-1-simplex

100% 21.6G/21.6G [12:49<00:00, 32.5MB/s]
100% 21.6G/21.6G [12:49<00:00, 30.1MB/s]


Modify and move the image and annotation files into the format YOLOv8 expects

In [ ]:
# Make the necessary folders
dataset_path = Path("datasets").joinpath("potholes")
dataset_path.mkdir(parents=True)

images_path = dataset_path.joinpath("images")
images_path.mkdir()

train_images_path = images_path.joinpath("train")
train_images_path.mkdir()

val_images_path = images_path.joinpath("val")
val_images_path.mkdir()

test_images_path = images_path.joinpath("test")
test_images_path.mkdir()

labels_path = dataset_path.joinpath("labels")
labels_path.mkdir()

train_labels_path = labels_path.joinpath("train")
train_labels_path.mkdir()

val_labels_path = labels_path.joinpath("val")
val_labels_path.mkdir()

test_labels_path = labels_path.joinpath("test")
test_labels_path.mkdir()

# Establish the current location of the data
kaggle_data = Path("Dataset 1 (Simplex)")
train_data = kaggle_data.joinpath("Train data")
test_data = kaggle_data.joinpath("Test data")

# Remove any duplicates
train_data_images = train_data.glob('**/*[0-9].JPG')
test_data_images = test_data.glob('**/*[0-9].JPG')

# Separate the testing data into testing and validation data
train_images, val_images = train_test_split([file for file in train_data_images])

# Move the images into their respective folders
for image in train_images:
  shutil.copy2(image, train_images_path)

for image in val_images:
  shutil.copy2(image, val_images_path)

for image in test_data_images:
  shutil.copy2(image, test_images_path)

# Create the regular expression needed to parse the training and validation annotations
# NOTE: Files are .JPGs, not .BMPs
p = re.compile(r"Train data\\Positive data\\(G[0-9]+).bmp ([0-9]+) (.*)")

# Extract the image file names
train_image_names = [image.name for image in train_images]
val_image_names = [image.name for image in val_images]

# Extract the annotations for the training and validation images
with open(kaggle_data.joinpath("simpleTrainFullPhotosSortedFullAnnotations.txt")) as f:
  for line in f.readlines():
    m = p.match(line)

    # Extract the image number of the current line
    name = f"{m.group(1)}.JPG"

    # Split the annotations into training and validation
    label_path = None
    if name in train_image_names:
      label_path = train_labels_path.joinpath(f"{m.group(1)}.txt")
    if name in val_image_names:
      label_path = val_labels_path.joinpath(f"{m.group(1)}.txt")
    with open(label_path, "w") as f:

      # Extract the number of bounding boxes
      count = int(m.group(2))

      # Extract the size and coordinates
      coords = [int(num) for num in m.group(3).strip().split(" ")]
      for i in range(0, count*4, 4):
        x_size = 3680
        y_size = 2760
        x = coords[i]
        y = coords[i + 1]
        width = coords[i + 2]
        height = coords[i + 3]

        # Convert the size and coordinates into the necessary format
        x_center = (x + width / 2) / x_size
        y_center = (y + height / 2) / y_size
        width = width / x_size
        height = height / y_size

        # Add the annotation to the annotation file
        f.write(f"0 {x_center} {y_center} {width} {height}\n")

# Create the regular expression needed to parse the testing annotations
# NOTE: Files are .JPGs, not .BMPs
p = re.compile(r"Test data\\(G[0-9]+).bmp ([0-9]+) (.*)")

# Extract the annotations for the testing images
with open(kaggle_data.joinpath("simpleTestFullSizeAllPotholesSortedFullAnnotation.txt")) as f:
  for line in f.readlines():
    m = p.match(line)
    label_path = test_labels_path.joinpath(f"{m.group(1)}.txt")
    with open(label_path, "w") as f:

      # Extract the number of bounding boxes
      count = int(m.group(2))

      # Extract the size and coordinates
      coords = [int(num) for num in m.group(3).strip().split(" ")]
      for i in range(0, count * 4, 4):
        x_size = 3680
        y_size = 2760
        x = coords[i]
        y = coords[i + 1]
        width = coords[i + 2]
        height = coords[i + 3]

        # Convert the size and coordinates into the necessary format
        x_center = (x + width / 2) / x_size
        y_center = (y + height / 2) / y_size
        width = width / x_size
        height = height / y_size

        # Add the annotation to the annotation file
        f.write(f"0 {x_center} {y_center} {width} {height}\n")

# Create the .yaml file YOLO uses to define the dataset
with open("potholes.yaml", "w") as f:
  f.write("path: potholes\n")
  f.write("train: images/train\n")
  f.write("val: images/val\n")
  f.write("test: images/test\n")
  f.write("names:\n")
  f.write("  0: pothole\n")

Perform hyperparameter tuning

In [ ]:
#model1 = ultralytics.YOLO('yolov8n.pt')
#result1 = model1.train(data='potholes.yaml', epochs=10, batch=-1, imgsz=1280, save_period=2, cache=True,
#                      plots=True, project="/content/drive/MyDrive/YOLO_Models", name="with_mosaic")

In [ ]:
#model2 = ultralytics.YOLO('yolov8n.pt')
#result2 = model2.train(data='potholes.yaml', epochs=10, batch=-1, imgsz=1280, save_period=2, cache=True,
#                      plots=True, project="/content/drive/MyDrive/YOLO_Models", name="without_mosaic", mosaic=0.0)

In [ ]:
#model3 = ultralytics.YOLO('yolov8n.pt')
#result3 = model3.train(data='potholes.yaml', epochs=10, batch=-1, imgsz=1280, save_period=2, cache=True,
#                      plots=True, project="/content/drive/MyDrive/YOLO_Models", name="some_mosaic", mosaic=0.4)

Train the final model

In [ ]:
model = ultralytics.YOLO('yolov8n.pt')
result = model.train(data='potholes.yaml', epochs=80, patience=10, batch=-1, imgsz=1280, save_period=5, cache=True, plots=True,
                     show_labels=False, project="/content/drive/MyDrive/YOLO_Models", name="80_epochs_10_patience", mosaic=0.4)

100%|██████████| 6.23M/6.23M [00:00<00:00, 25.1MB/s]


Ultralytics YOLOv8.2.2 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=potholes.yaml, epochs=80, time=None, patience=10, batch=-1, imgsz=1280, save=True, save_period=5, cache=True, device=None, workers=8, project=/content/drive/MyDrive/YOLO_Models, name=80_epochs_10_patience, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=Fals

100%|██████████| 755k/755k [00:00<00:00, 5.15MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/datasets/potholes/labels/train... 972 images, 2536 backgrounds, 0 corrupt: 100%|██████████| 3508/3508 [00:03<00:00, 1159.97it/s]


...

      80/80      10.2G      1.626      1.061      1.135         24       1280: 100%|██████████| 195/195 [00:48<00:00,  4.05it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [00:05<00:00,  6.21it/s]

                   all       1180       1040      0.712      0.585      0.658      0.321



80 epochs completed in 1.494 hours.
Optimizer stripped from /content/drive/MyDrive/YOLO_Models/80_epochs_10_patience/weights/last.pt, 6.4MB
Optimizer stripped from /content/drive/MyDrive/YOLO_Models/80_epochs_10_patience/weights/best.pt, 6.4MB

Validating /content/drive/MyDrive/YOLO_Models/80_epochs_10_patience/weights/best.pt...
Ultralytics YOLOv8.2.2 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [00:18<00:00,  1.79it/s]


                   all       1180       1040      0.735      0.593      0.665      0.321
Speed: 1.5ms preprocess, 4.4ms inference, 0.0ms loss, 4.6ms postprocess per image
Results saved to /content/drive/MyDrive/YOLO_Models/80_epochs_10_patience


Perform predictions on the test set

In [ ]:
model = ultralytics.YOLO("/content/drive/MyDrive/YOLO_Models/80_epochs_10_patience/weights/best.pt")
testing_result = model.val(data='potholes.yaml', imgsz=1280, plots=True, split="test")

Ultralytics YOLOv8.2.2 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/datasets/potholes/labels/test.cache... 650 images, 0 backgrounds, 0 corrupt: 100%|██████████| 650/650 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 41/41 [00:27<00:00,  1.52it/s]


                   all        650       1352      0.777       0.66      0.736      0.371
Speed: 0.6ms preprocess, 4.7ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/val2


Calulate the F1-score

In [ ]:
testing_result.box.f1[0]

0.7134536061227534

Perform a prediction on a particularly difficult image

In [ ]:
test_image = test_images_path.joinpath("G0011476.JPG")
prediction = model.predict(source=test_image, visualize=True, imgsz=1280, save=True, show_labels=False)


Saving runs/detect/predict/G0011476/stage0_Conv_features.png... (16/16)
Saving runs/detect/predict/G0011476/stage1_Conv_features.png... (32/32)
Saving runs/detect/predict/G0011476/stage2_C2f_features.png... (32/32)
Saving runs/detect/predict/G0011476/stage3_Conv_features.png... (32/64)
Saving runs/detect/predict/G0011476/stage4_C2f_features.png... (32/64)
Saving runs/detect/predict/G0011476/stage5_Conv_features.png... (32/128)
Saving runs/detect/predict/G0011476/stage6_C2f_features.png... (32/128)
Saving runs/detect/predict/G0011476/stage7_Conv_features.png... (32/256)
Saving runs/detect/predict/G0011476/stage8_C2f_features.png... (32/256)
Saving runs/detect/predict/G0011476/stage9_SPPF_features.png... (32/256)
Saving runs/detect/predict/G0011476/stage10_Upsample_features.png... (32/256)
Saving runs/detect/predict/G0011476/stage11_Concat_features.png... (32/384)
Saving runs/detect/predict/G0011476/stage12_C2f_features.png... (32/128)
Saving runs/detect/predict/G0011476/stage13_Upsampl

Save the predictions on the test set and the individual image to avoid losing them after disconnecting from the runtime

In [ ]:
!cp -r runs drive/MyDrive/YOLO_Models/80_epochs_10_patience/

Export the model in the desired format

In [ ]:
model.export(format='ncnn')

Ultralytics YOLOv8.2.2 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)

PyTorch: starting from '/content/drive/MyDrive/YOLO_Models/80_epochs_10_patience/weights/best.pt' with input shape (1, 3, 1280, 1280) BCHW and output shape(s) (1, 5, 33600) (6.1 MB)

TorchScript: starting export with torch 2.2.1+cu121...
TorchScript: export success ✅ 2.6s, saved as '/content/drive/MyDrive/YOLO_Models/80_epochs_10_patience/weights/best.torchscript' (12.2 MB)
requirements: Ultralytics requirement ['ncnn'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 6.2 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 7.4s, installed 1 package: ['ncnn']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


NCNN: starting export with NCNN 1.0.20240410...
NCNN: WARNING ⚠️ PNNX not found. Attempting to download binary file from https://github.com/pnnx/pnnx/.
Note PNNX Binary file must be placed in current working directory or in /us

100%|██████████| 17.3M/17.3M [00:00<00:00, 42.5MB/s]
Unzipping pnnx-20240410-linux.zip to /content/pnnx-20240410-linux...: 100%|██████████| 3/3 [00:00<00:00,  8.34file/s]

NCNN: running '/usr/local/lib/python3.10/dist-packages/ultralytics/pnnx /content/drive/MyDrive/YOLO_Models/80_epochs_10_patience/weights/best.torchscript ncnnparam=/content/drive/MyDrive/YOLO_Models/80_epochs_10_patience/weights/best_ncnn_model/model.ncnn.param ncnnbin=/content/drive/MyDrive/YOLO_Models/80_epochs_10_patience/weights/best_ncnn_model/model.ncnn.bin ncnnpy=/content/drive/MyDrive/YOLO_Models/80_epochs_10_patience/weights/best_ncnn_model/model_ncnn.py pnnxparam=/content/drive/MyDrive/YOLO_Models/80_epochs_10_patience/weights/best_ncnn_model/model.pnnx.param pnnxbin=/content/drive/MyDrive/YOLO_Models/80_epochs_10_patience/weights/best_ncnn_model/model.pnnx.bin pnnxpy=/content/drive/MyDrive/YOLO_Models/80_epochs_10_patience/weights/best_ncnn_model/model_pnnx.py pnnxonnx=/content/drive/MyDrive/YOLO_Models/80_epochs_10_patience/weights/best_ncnn_model/model.pnnx.onnx fp16=0 device=cpu inputshape="[1, 3, 1280, 1280]"'


NCNN: export success ✅ 14.9s, saved as '/content/drive/MyDrive/YOLO_Models/80_epochs_10_patience/weights/best_ncnn_model' (12.1 MB)

Export complete (19.6s)
Results saved to /content/drive/MyDrive/YOLO_Models/80_epochs_10_patience/weights
Predict:         yolo predict task=detect model=/content/drive/MyDrive/YOLO_Models/80_epochs_10_patience/weights/best_ncnn_model imgsz=1280  
Validate:        yolo val task=detect model=/content/drive/MyDrive/YOLO_Models/80_epochs_10_patience/weights/best_ncnn_model imgsz=1280 data=potholes.yaml  
Visualize:       https://netron.app


'/content/drive/MyDrive/YOLO_Models/80_epochs_10_patience/weights/best_ncnn_model'

**Citations**

Nienaber, S & Booysen, M.J. (Thinus) & Kroon, Rs. (2015). Detecting Potholes Using Simple Image Processing Techniques and Real-World Footage. 10.13140/RG.2.1.3121.8408.

Nienaber, S. (2016). Detecting potholes with monocular computer vision: A Performance evaluation of techniques.

```
@software{yolov8_ultralytics,
  author = {Glenn Jocher and Ayush Chaurasia and Jing Qiu},
  title = {Ultralytics YOLOv8},
  version = {8.0.0},
  year = {2023},
  url = {https://github.com/ultralytics/ultralytics},
  orcid = {0000-0001-5950-6979, 0000-0002-7603-6750, 0000-0003-3783-7069},
  license = {AGPL-3.0}
}
```